# 応用ユースケース：金融機関と高級時計会社のクロス集計分析

このチュートリアルでは、AutoPrivacy DCRを使用して、金融機関（銀行）と高級時計会社が保持するデータを安全に突合し、顧客の金融属性と購買行動を分析する応用例を紹介します。

**重要**: このチュートリアルは、[基本チュートリアル](./basic_apc_cli_tutorial.ipynb)の実行手順を理解していることを前提としています。APC-CLIの具体的なコマンドや実行方法については基本チュートリアルを参照してください。

---

## ユースケースの背景

企業が単独で保有するデータには限界があります：

- **銀行** → 顧客の「収入・資産・信用度」などの金融的な側面は分かるが、**具体的にどんなモノにお金を使っているか**までは分からない
- **高級時計会社** → 「どの顧客が時計を買ったか」「どのラインに興味があるか」は分かるが、その顧客の**収入水準や資産規模、信用力**は分からない

## 解決策

**AutoPrivacy DCR上でIDを元に安全に同意なしで突合することで「金融属性 × 購買実績」のクロス集計が可能になり、マーケティングROIを高められる**

異なる企業のデータを組み合わせることで、単独では見えなかった顧客の全体像が明らかになります。

## データ概要

### 銀行データ（入力1）
- **account_balance**: 口座残高レベル (High/Medium/Low)
- **monthly_income**: 月収レベル (High/Medium/Low) 
- **credit_score**: 信用スコア (High/Medium/Low)

### 高級時計会社データ（入力2）
- **total_purchase_amount**: 累計購入額 (High/Medium/Low)
- **last_purchase_date**: 最終購入日からの経過 (Within_1year/1-3years/Over_3years)
- **store_visits_12months**: 過去12か月の来店回数 (High/Medium/Low)
- **product_category_interest**: 関心商品カテゴリー (Luxury/Classic/Sports)

### 期待される分析結果
- どの金融属性の顧客がどの購買パターンを持つか
- 来店頻度と購買額の相関
- 商品嗜好と顧客属性の関係
- 潜在的な高価値顧客の発見

## クロス集計表アルゴリズムの概要

クロス集計表とは、2つ以上のカテゴリカル変数（質的変数）の関係を表形式で表現する統計手法です。このチュートリアルでは、銀行と高級時計会社の2つのデータセットをIDで突合し、クロス集計表を作成します。

<img src="../img/cross_table_overview.svg" width=600>

**処理の流れ**：
1. 両組織が同一の顧客IDを持つレコードを特定
2. IDが一致するレコード同士を安全に結合
3. 結合されたデータから各属性の組み合わせをカウント
4. カウント結果をクロス集計表として出力

---

## 実行方法

このユースケースの実行手順は、[基本チュートリアル](./basic_apc_cli_tutorial.ipynb)と同じです。以下の点のみ変更してください：

**必須の変更:**
- **関数のパス**: `FUNCTION_SOURCE_PATH=../../functions/cross_table`
- **入力データのパス**:
  - `INPUT_1_PATH=../../functions/cross_table/inputs/bank_input`（銀行データ）
  - `INPUT_2_PATH=../../functions/cross_table/inputs/luxury_watch_input`（高級時計会社データ）
- **handler.pyで読み込むファイル名**: `functions/cross_table/function/handler.py`内の入力ファイル名を以下のように変更：
  ```python
  # 変更前
  df_cross_table = cross_table_data(
      f"{INPUT_1_DIR}/input_1.csv", f"{INPUT_2_DIR}/input_2.csv"
  )
  
  # 変更後
  df_cross_table = cross_table_data(
      f"{INPUT_1_DIR}/data.csv", f"{INPUT_2_DIR}/data.csv"
  )
  ```

**任意の変更（わかりやすくするため変更を推奨）:**
- **プロファイル名**: 例として `bank-profile`（銀行用）と `luxury-watch-profile`（高級時計会社用）
- **アプリケーション名**: 例として `cross_table_analysis`

基本チュートリアルのステップ3〜15に従って、環境変数の設定からクリーンアップまでを実行してください。

---

## 突合結果の例

基本チュートリアルの手順に従って実行すると、以下のようなクロス集計結果が得られます：

```
number_of_rows,0:account_balance,0:credit_score,0:monthly_income,1:last_purchase_date,1:product_category_interest,1:store_visits_12months,1:total_purchase_amount
11,Low,High,High,Over_3years,Sports,High,Low
10,Low,Low,Medium,1-3years,Sports,Low,Low
12,High,Medium,High,Over_3years,Sports,High,High
10,Low,Low,Medium,1-3years,Classic,Low,Low
10,Medium,High,Medium,1-3years,Classic,High,Medium
10,Low,Low,Low,Within_1year,Sports,High,Low
12,Low,Medium,Medium,1-3years,Classic,High,Low
11,Medium,Medium,High,Over_3years,Luxury,Low,Medium
11,Low,High,High,Over_3years,Classic,High,Low
13,Medium,Low,High,Over_3years,Sports,High,Medium
10,Medium,High,Low,Within_1year,Luxury,Low,Medium
10,Low,Medium,High,Over_3years,Classic,High,Low
11,Low,High,Medium,1-3years,Sports,High,Low
10,Medium,High,Medium,1-3years,Classic,Low,Medium
10,Medium,Low,High,Over_3years,Classic,High,Medium
10,Low,Medium,High,Over_3years,Luxury,Low,Low
11,High,High,Low,Within_1year,Luxury,High,High
13,High,High,Medium,1-3years,Luxury,High,High
```

**データの見方:**
- `number_of_rows`: この属性の組み合わせを持つ顧客数
- `0:` で始まるカラム: 銀行データ（口座残高、信用スコア、月収）
- `1:` で始まるカラム: 高級時計会社データ（購買履歴、来店頻度、関心カテゴリー）

例えば、1行目は「口座残高Low、信用スコアHigh、月収Highで、3年以上前に購入し、Sports志向で来店頻度がHighだが購買額はLowの顧客が11人いる」ことを示しています。

---

## 分析結果の考察

クロス集計分析から得られる代表的な知見：

### 1. **来店は多いが購買額は控えめな層が存在**

→ よく店舗を訪れるものの、購買に至るケースは控えめという傾向が見られる顧客セグメント。

**アクション例**: 
- 試着体験やエントリーモデルの提案を充実させ、購買につなげる
- 来店ポイントプログラムで関係性を強化

### 2. **嗜好と購買行動に多様性がある**

→ スポーティなモデルを好む顧客の中には、購買額が安定しない傾向の方もいる。嗜好と購買行動の差異を理解することで、プロモーションや商品ラインナップの工夫が有効。

**アクション例**:
- 嗜好に応じた価格帯別商品展開
- スポーツモデル愛好者向けアクセサリー提案

### 3. **金融資産や信用が購買に影響するケース**

→ 収入水準に関わらず、資産や信用力がある顧客は高額購買につながるケースが確認できる。所得だけでなく、資産・信用の観点から顧客を理解することが重要。

**アクション例**:
- 金融資産・信用力を基にしたVIPプログラム
- 限定招待イベントでの接点作り

### 4. **Luxury志向は幅広い層に分布**

→ 高収入層だけでなく、中間層やその他層にも高級志向のニーズが見られる。高級志向は必ずしも所得や資産と連動せず、幅広い顧客に潜在ニーズが存在する。

**アクション例**:
- 「手の届くラグジュアリー」商品展開
- 将来的な上位モデルへのステップアッププログラム

---

## 他の応用ユースケース

このクロス集計分析フレームワークは、様々なドメインの組織間データ突合に応用可能です：

### 1. **銀行 × 小売・EC（消費行動突合）**

- **目的**: 購買力・嗜好・来店頻度を把握して、パーソナライズマーケティング
- **例**:
  - 銀行口座残高・クレジット利用履歴 × スーパー／コンビニの購買履歴
  - クレジットカード支払い履歴 × オンラインファッション通販の購入履歴
- **得られる知見**: 「高収入だけど頻度は少ない」「低収入でも嗜好が偏っている」など顧客セグメントの可視化

### 2. **フィットネスジム × 健康管理アプリ**

- **目的**: 顧客の健康行動と利用状況の連携
- **例**:
  - ジム来店頻度 × 歩数や心拍データ
  - クラス予約履歴 × 栄養管理アプリの摂取カロリーデータ
- **得られる知見**: 「ジムは来ているけど運動量は少ない層」「自己管理はしているがジムに来ない層」など

### 3. **通販 × ソーシャルメディア嗜好**

- **目的**: 広告ターゲティングや商品開発
- **例**:
  - 購買履歴 × InstagramやTikTokのフォロー・いいね履歴
  - ECサイトの閲覧履歴 × YouTube動画視聴嗜好
- **得られる知見**: 「ブランド好きだが購買にはつながっていない層」「購入傾向がSNSで予測できる層」

### 4. **旅行会社 × 交通系カード**

- **目的**: 顧客の旅行嗜好と消費能力の把握
- **例**:
  - 航空券・ホテル予約履歴 × クレジットカード利用履歴
  - 移動距離（Suica/Pasmo等） × ホテル滞在履歴
- **得られる知見**: 「旅行好きだけど予算が低い層」「高額旅行も余裕で楽しめる層」

### 5. **飲食チェーン × 健康データ**

- **目的**: ヘルシー志向・購買行動の把握
- **例**:
  - 来店履歴・注文履歴 × ウェアラブルの食事・歩数データ
  - サブスクリプション利用履歴 × 体重・BMI推移
- **得られる知見**: 「健康意識は高いが頻度は少ない層」「頻度は多いが偏った注文傾向の層」

---

**まとめ**

異なるドメインを突合することで **隠れた顧客セグメントや意外な行動パターン**が見えてきます。AutoPrivacy DCRを活用することで、プライバシーを保護しながら、これまで見えなかった顧客インサイトを発見できます。